<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/experiment/%E5%8F%A6%E4%B8%80%E7%A8%AE%E6%8E%A8%E8%96%A6%E6%96%B9%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2300, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 2300 (delta 12), reused 0 (delta 0), pack-reused 2270
Receiving objects: 100% (2300/2300), 63.92 MiB | 21.57 MiB/s, done.
Resolving deltas: 100% (1505/1505), done.


# import

In [2]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# demo data

In [3]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
118-017,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,72.0,0.2127,35.413666,57.973106,80.0,920.0,0.01,5.000005,36.0


# 這一筆樣本的NA in Benzene_ppmw

In [4]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
118-017,920.0


# 這一筆樣本的Benzene in C620 Sidedraw_wt%

In [ ]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
118-017,80.0


# 這一筆樣本的Toluene in Benzene_ppmw

In [5]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
118-017,5.000005


# 1.利用兩次試算來達成推薦效果

In [16]:
f = joblib.load('/content/c620/model/allsystem.pkl')
demo = joblib.load('./data/demo.pkl')
# 第一次試算時原本的數值為 [920,80,5]
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])
# 把數值替換成 [980,70,10]
demo['icg_input']['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = 980 
demo['icg_input']['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = 70
demo['icg_input']['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = 10
# 第二次試算數值就是 [980,70,10]
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2 = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])
# 計算調幅
c620_op_Δ = c620_op2 - c620_op1
c660_op_Δ = c660_op2 - c660_op1
c670_op_Δ = c670_op2 - c670_op1

In [17]:
# 由於Benzene in C620 Sidedraw_wt% 從 80 下降到 70 根據化工知識以下兩個溫度上升很合理也很合邏輯
c620_op_Δ.iloc[:,-2:]

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
118-017,0.680298,0.468353


In [18]:
c660_op_Δ.iloc[:,-2:]

,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
118-017,-0.01741,0.297371


In [19]:
c670_op_Δ.iloc[:,-2:]

,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
118-017,-0.112793,-0.127335
